<a href="https://colab.research.google.com/github/rakshit9/Generative-AI/blob/main/LangGraph_Article_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:

!pip -q install "langchain>=0.3.0" "langgraph>=0.2.0" "langchain-openai>=0.2.0" "python-dotenv>=1.0.1"

In [31]:
from google.colab import userdata

import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [32]:
from dataclasses import dataclass, field
from typing import List, Dict
from openai import OpenAI

@dataclass
class ArticleState:
    topic: str = ""
    notes: list = field(default_factory=list)
    outline: list = field(default_factory=list)
    sections: list = field(default_factory=list)
    article: str = ""

out_dict = app.invoke({"topic": "LangGraph 101", "notes": ["graphs", "state", "nodes"]})
out = ArticleState(**out_dict)   # now you can do out.outline, out.article
def llm(prompt: str) -> str:
    """Simple LLM helper (stub if no key)."""
    if not os.getenv("OPENAI_API_KEY"):
        # fake LLM output just to visualize the flow
        return f"[stub output for prompt → {prompt[:100]}...]"
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":prompt}],
        temperature=0.4,
        max_tokens=400,
    )
    return resp.choices[0].message.content.strip()

In [33]:
from langgraph.graph import StateGraph, END

def node_outline(state: ArticleState) -> ArticleState:
    prompt = f"Create 3–5 section titles for an article about '{state.topic}' using these notes:\n{state.notes}"
    raw = llm(prompt)
    state.outline = [ln.strip("-• ").strip() for ln in raw.splitlines() if ln.strip()][:5]
    return state

def node_sections(state: ArticleState) -> ArticleState:
    sections = []
    for title in state.outline:
        prompt = f"Write 2 short paragraphs for section '{title}' about {state.topic}."
        body = llm(prompt)
        sections.append({"title": title, "body": body})
    state.sections = sections
    return state

def node_assemble(state: ArticleState) -> ArticleState:
    article_parts = [f"# {state.topic}"]
    for s in state.sections:
        article_parts.append(f"## {s['title']}\n{s['body']}")
    state.article = "\n\n".join(article_parts)
    return state


In [34]:
# ✅ Minimal LangGraph demo using dict state (no attribute access)

from langgraph.graph import StateGraph, END

# --- "LLM" stub (replace with your real call if you have a key) ---
def llm(prompt: str) -> str:
    return f"[stub] {prompt[:120]}..."

# --- Nodes expect/return dicts ---
def node_make_outline(state: dict) -> dict:
    raw = llm(f"Create 3–5 section titles for '{state['topic']}' using notes: {state['notes']}")
    outline = [ln.strip("-• ").strip() for ln in raw.splitlines() if ln.strip()][:5]
    state["outline"] = outline if outline else ["Introduction", "Key Ideas", "Conclusion"]
    return state

def node_draft_sections(state: dict) -> dict:
    sections = []
    for title in state["outline"]:
        body = llm(f"Write 2 short paragraphs for section '{title}' about {state['topic']}.")
        sections.append({"title": title, "body": body})
    state["sections"] = sections
    return state

def node_assemble_article(state: dict) -> dict:
    parts = [f"# {state['topic']}"]
    for s in state["sections"]:
        parts.append(f"## {s['title']}\n{s['body']}")
    state["article"] = "\n\n".join(parts)
    return state

# --- Build graph ---
graph = StateGraph(dict)
graph.add_node("make_outline", node_make_outline)
graph.add_node("draft_sections", node_draft_sections)
graph.add_node("assemble_article", node_assemble_article)

graph.set_entry_point("make_outline")
graph.add_edge("make_outline", "draft_sections")
graph.add_edge("draft_sections", "assemble_article")
graph.add_edge("assemble_article", END)

app = graph.compile()

# --- Run with dict input ---
state_in = {
    "topic": "How LangGraph Helps Build AI Agents",
    "notes": [
        "Workflows as graphs (nodes + edges).",
        "Single state object flows through nodes.",
        "Great for multi-step LLM apps.",
    ],
}
result = app.invoke(state_in)

# --- Access dict keys (not attributes) ---
print("OK! Outline:", result["outline"][:5])
print("\nARTICLE PREVIEW:\n", result["article"][:600])


OK! Outline: ["[stub] Create 3–5 section titles for 'How LangGraph Helps Build AI Agents' using notes: ['Workflows as graphs (nodes + edges).'..."]

ARTICLE PREVIEW:
 # How LangGraph Helps Build AI Agents

## [stub] Create 3–5 section titles for 'How LangGraph Helps Build AI Agents' using notes: ['Workflows as graphs (nodes + edges).'...
[stub] Write 2 short paragraphs for section '[stub] Create 3–5 section titles for 'How LangGraph Helps Build AI Agents' using n...
